In [5]:
import gzip
import json
import pandas as pd
import re
import ast
import requests
from textblob import TextBlob
from io import BytesIO
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MultiLabelBinarizer

In [15]:
'''URL a los datos en Github '''
horas_jugadas_parquet = 'https://github.com/rcarmonae/HENRY_PI01_RCARMONA_MAYO2024/raw/main/DATASETS/F01_PlaytimeGenre.parquet'
horas_jugadas_usuario_parquet = 'https://github.com/rcarmonae/HENRY_PI01_RCARMONA_MAYO2024/raw/main/DATASETS/F02_UserForGenre.parquet'
game_reviews_parquet = 'https://github.com/rcarmonae/HENRY_PI01_RCARMONA_MAYO2024/raw/main/DATASETS/F03_UsersRecommend.parquet'


'''Solicita el contenido de las URL'''
res_horas_jugadas = requests.get(horas_jugadas_parquet)
res_horas_jugadas_usuario = requests.get(horas_jugadas_usuario_parquet)
res_game_reviews = requests.get(game_reviews_parquet)


'''Convierte los csv a dataframe'''
horas_jugadas = pd.read_parquet(BytesIO(res_horas_jugadas.content))
horas_jugadas_usuario = pd.read_parquet(BytesIO(res_horas_jugadas_usuario.content))
game_reviews = pd.read_parquet(BytesIO(res_game_reviews.content))

In [16]:
'''FUNCIÓN 1: PlaytimeGenre'''
### lee la base de datos necesaria
#horas_jugadas = pd.read_parquet(r'G:\My Drive\HENRY\PROYECTO_INDIVIDUAL_I\PI01-ABRIL2024\DATASET\F01_PlaytimeGenre.parquet')

genre = 'RPG'
# MODIFICAR ESTO PARA QUE LLAME AL ARGUMENTO DE LA FUNCIÓN  - Filtrar por un género específico, digamos RPG
horas_jugadas_filtrado = horas_jugadas[horas_jugadas['genres'] == genre]
# filtered_data = horas_jugadas_genre[horas_jugadas_2018['genres'] != 'Early Access']

# Agrupar por años y sumar las horas jugadas de cada uno
total_playtime_year = horas_jugadas_filtrado.groupby('release_year')['playtime_forever'].sum()

# Encontrar el año con el mayor tiempo jugado para el género dado
year_most_played = total_playtime_year.idxmax()

# Modificar para que diga el año del ARGUMENTO de la función
print(f"Año de lanzamiento con más horas jugadas para el género {genre}: {int(year_most_played)}")

Año de lanzamiento con más horas jugadas para el género RPG: 2011


In [17]:
'''FUNCIÓN 2: UserForGenre'''
### lee la base de datos necesaria
#horas_jugadas_usuario = pd.read_parquet(r'G:\My Drive\HENRY\PROYECTO_INDIVIDUAL_I\PI01-ABRIL2024\DATASET\F02_UserForGenre.parquet')

genre = 'RPG'

# MODIFICAR ESTO PARA QUE LLAME AL ARGUMENTO DE LA FUNCIÓN  - Filtrar por un género específico, digamos RPG
horas_jugadas_usuario_filtrado = horas_jugadas_usuario[horas_jugadas_usuario['genres'] == genre]

# Agrupar por años y sumar las horas jugadas de cada uno
total_playtime_user = horas_jugadas_usuario_filtrado.groupby(['user_id','release_year'])['playtime_forever'].sum()

# Agrupar otra vez por usuario
user_group =total_playtime_user.groupby('user_id').sum()

# Encontrar el jugador con el mayor tiempo jugado para el género dado
user_most_played = user_group.idxmax()

# Segmenta el DF con el resumen de usuario y año, para obtener solo los registros del usuario que más horas jugó 
user_years = total_playtime_user.loc[user_most_played]

# Prepara el dataframe para imprimirlo en el formato adecuado
user_years = user_years.reset_index() # Genera un nuevo índice para liberar el año del registro
user_years['release_year'] = user_years['release_year'].astype(int) # Convierte los años de float a int
user_years['playtime_forever'] = user_years['playtime_forever'].astype(int) # Convierte las horas jugadas de float a int
user_years = user_years[user_years['playtime_forever'] != 0] # Remueve las filas de los años en los que las horas jugadas = 0
list_of_dicts = user_years.to_dict('records') # Convierte el DataFrame a una lista de diccionarios
formatted_entries = [f"{{Año: {d['release_year']}, Horas: {d['playtime_forever']}}}" for d in list_of_dicts] # Crear una lista de strings formateados como diccionarios
formatted_string = ", ".join(formatted_entries) # Unir la lista en un string único

# Formatear el mensaje para imprimir
final_message = f"Usuario con más horas jugadas para el género {genre}: {user_most_played}, Horas jugadas: [{formatted_string}]"

print(final_message)

Usuario con más horas jugadas para el género RPG: shinomegami, Horas jugadas: [{Año: 1999, Horas: 377}, {Año: 2003, Horas: 530943}, {Año: 2004, Horas: 696}, {Año: 2005, Horas: 185}, {Año: 2006, Horas: 2023}, {Año: 2007, Horas: 135020}, {Año: 2008, Horas: 1305}, {Año: 2009, Horas: 1382}, {Año: 2010, Horas: 4311}, {Año: 2011, Horas: 17752}, {Año: 2012, Horas: 12819}, {Año: 2013, Horas: 107775}, {Año: 2014, Horas: 81938}, {Año: 2015, Horas: 111419}, {Año: 2016, Horas: 15550}]


In [5]:
'''FUNCIÓN 3: UsersRecommend'''
# ESTA FUNCIÓN PARTE DEL DT game_reviews
game_reviews = pd.read_parquet(r'G:\My Drive\HENRY\PROYECTO_INDIVIDUAL_I\PI01-ABRIL2024\DATASET\F03_UsersRecommend.parquet')
def top_recommended_games(year):
    # Filtrar el DataFrame por año, recomendación positiva y sentimientos positivos/neutrales
    filtered_df = game_reviews[
        (game_reviews['release_year'] == year) & 
        (game_reviews['recommend'] == True) & 
        (game_reviews['sentiment_analysis'] >= 1)
    ]
    
    # Contar las recomendaciones por juego
    recommendation_counts = filtered_df['item_id'].value_counts().head(3)
    
    # Crear un DataFrame para los resultados
    top_games = pd.DataFrame({
        'item_id': recommendation_counts.index,
        'recommendations': recommendation_counts.values      
    })
    
    # join entre top games y filtered_df para asignar los nombres (titulos) de los items
    top_game_titles = pd.merge(top_games[['item_id','recommendations']], filtered_df[['item_id','title']], left_on='item_id', right_on='item_id', how='left')
    top_game_titles = top_game_titles.drop_duplicates(subset=['item_id'])
    top_game_titles.reset_index(drop=True, inplace=True)
    final_message = f"[Puesto 1: {top_game_titles.loc[0,'title']}, Puesto 2: {top_game_titles.loc[1,'title']}, Puesto 3: {top_game_titles.loc[2,'title']}]"
    return final_message


# Obtener el top 3 de juegos recomendados para el año 2021
top_games_1995 = top_recommended_games(1996)
print(top_games_1995)


[Puesto 1: Pajama Sam: No Need to Hide When It's Dark Outside, Puesto 2: I Have No Mouth, and I Must Scream, Puesto 3: The Original Strife: Veteran Edition]


In [40]:
# Supongamos que 'df' es el DataFrame global que contiene la información de los juegos
df = pd.read_parquet(r'G:\My Drive\HENRY\PROYECTO_INDIVIDUAL_I\PI01-ABRIL2024\DATASET\F03_UsersRecommend.parquet')
df['item_id'] = df['item_id'].astype(int)

def recommend_games(item_id):
    global df  # Utiliza el DataFrame global

    # Asegúrate de que los géneros estén correctamente formateados como listas
    if isinstance(df.loc[0, 'genres'], str):
        df['genres'] = df['genres'].apply(eval)

    try:
        game_info = df[df['item_id'] == item_id].iloc[0]
    except IndexError:
        return "No se encontró el juego con el item_id proporcionado."

    mlb = MultiLabelBinarizer()
    genre_encoded = mlb.fit_transform(df['genres'])
    genre_df = pd.DataFrame(genre_encoded, columns=mlb.classes_, index=df.index)

    features = pd.concat([df[['sentiment_analysis', 'recommend', 'release_year']], genre_df], axis=1)
    features.dropna(inplace=True)

    # Eliminar duplicados para asegurar que cada juego es único
    features = features[~df['item_id'].duplicated(keep='first')]

    # Almacenar los índices antes de eliminar el juego dado para posterior referencia
    valid_indices = features.index

    target_features = features.loc[df['item_id'] == item_id]
    features = features.drop(index=target_features.index)

    # Calcular la similitud del coseno entre el juego dado y todos los otros juegos
    cosine_sim = cosine_similarity(features, target_features)

    # Asignar los puntajes de similitud al DataFrame que solo contiene los índices válidos
    valid_df = df.loc[valid_indices].drop(index=target_features.index)
    valid_df['similarity_score'] = cosine_sim[:, 0]

    # Asegurar que las recomendaciones sean juegos únicos
    valid_df = valid_df.groupby('item_id').max('similarity_score')

    return valid_df.nlargest(5, 'similarity_score')

# Suponiendo que df es tu DataFrame ya definido y quieres recomendar juegos basados en un item_id específico
item_id = 1250  # Asegúrate de usar un item_id válido
recommended_games = recommend_games(item_id)
print(recommended_games)





         sentiment_analysis  recommend  release_year  similarity_score
item_id                                                               
9480                      2       True        2009.0               1.0
20500                     2       True        2009.0               1.0
21660                     2       True        2009.0               1.0
35420                     2       True        2009.0               1.0
300                       2       True        2010.0               1.0


C:\Users\occar\AppData\Local\Temp\ipykernel_33652\2478314285.py:29: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  features = features[~df['item_id'].duplicated(keep='first')]
